# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # insider db
# wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')

In [ ]:
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')

# Load

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [6]:
# exasol db
# 8.6s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")

In [7]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
}

In [8]:
carr = car.rename(columns=car_ch_col)

In [9]:
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [10]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")

In [11]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [12]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [13]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_NM  46333 non-null  object
 2   STDG_SGG_NM   46310 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [14]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [15]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [16]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

In [17]:
coder.head()

,법정동코드,시도,시군구
0,4171004506,경기도,양주군
1,4171004507,경기도,양주군
2,4171004508,경기도,양주군
3,4171004509,경기도,양주군
4,4171005000,경기도,양주군


## 노후차 조기폐차 관리정보(수도권)(STD_DLM_TB_ERP_EARLY_ERASE_AEA)

In [18]:
# 2.4s
aea = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
aea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804082 entries, 0 to 804081
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  804082 non-null  object
 1   ELPDSRC_STTS_CD      804082 non-null  object
 2   ELPDSRC_LST_APRV_YN  683874 non-null  object
dtypes: object(3)
memory usage: 18.4+ MB


In [19]:
aea.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [20]:
aea_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [21]:
aear = aea.rename(columns=aea_ch_col)
aear.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

In [22]:
aear.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN
0,KMCXLS7BP2U626435,A32Y,Y
1,KNAJA5265VA794691,A32Y,Y
2,KPBLC3D81VP018295,A32Y,Y
3,KPDHADMD1YP225493,A32Y,Y
4,KNAMA75122S249083,A32Y,Y


## 노후차 조기폐차 관리정보(수도권외)(STD_DLM_TB_ERP_EARLY_ERASE_LGV)

In [23]:
# 1.8s
lgv = wd.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_LGV;")
lgv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373371 entries, 0 to 373370
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  373371 non-null  object
 1   ELPDSRC_STTS_CD      373371 non-null  object
 2   ELPDSRC_LST_APRV_YN  160690 non-null  object
dtypes: object(3)
memory usage: 8.5+ MB


In [24]:
lgv.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [25]:
lgv_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [26]:
lgvr = lgv.rename(columns=lgv_ch_col)
lgvr.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

In [27]:
lgvr.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN
0,KMHSD81VP4U791871,A32M,NaN
1,KPBFA2AF16P223800,A32M,NaN
2,KMFWVH7HP7U772891,A32M,NaN
3,KMFXKN7BPYU361960,A32M,NaN
4,KMFXKS7BP2U563724,A32M,NaN


# 전처리

## 중복 차대번호 제거

In [28]:
carr.shape, len(carr['차대번호'].unique())

((1898814, 10), 1898763)

In [29]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1898763, 10)

## 등록정보 말소 제거

In [30]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1515736, 10)

## 차종 코드 변환

In [31]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carm['차종'] = carm['차종'].replace(cd_dict)
carm['차종'].unique()

array(['화물', '승용', '승합', '특수', nan], dtype=object)

## 용도 코드 변환

In [32]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carm['용도'] = carm['용도'].replace(purps_dict)
carm['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 연료 코드 변환

In [33]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', '태양열', 'CNG 하이브리드', '등유', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [34]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [35]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1515736, 16)

## 지역 정보 부착

In [36]:
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

In [37]:
csc = cs.merge(coder, on='법정동코드', how='left')
csc.shape

(1515736, 18)

In [38]:
csc['시도'].isnull().sum()

1

In [39]:
# 경기도 양주시
csc.loc[csc['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

In [40]:
csc['시도'].isnull().sum()

0

## 조기폐차 정보들 병합

In [41]:
elp = pd.concat([aear, lgvr], ignore_index=True)
elp.shape

(1177453, 3)

In [42]:
elp.shape, len(elp['차대번호'].unique())

((1177453, 3), 1105195)

In [43]:
elpm = elp.sort_values('조기폐차최종승인YN', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elpm.shape

(1105195, 3)

In [44]:
elpm = elpm[elpm['조기폐차최종승인YN'] == 'Y'].reset_index(drop=True)
elpm.shape

(844474, 3)

## 조기폐차 정보 추가

In [45]:
csc.shape

(1515736, 18)

In [46]:
df = csc.merge(elpm, on='차대번호', how='left')
df.shape

(1515736, 20)

In [47]:
df['연료'].value_counts(dropna=False)

연료
경유             1156240
휘발유             333205
LPG(액화석유가스)      26179
NaN                 71
기타연료                39
CNG(압축천연가스)          1
알코올                  1
Name: count, dtype: int64

In [48]:
df = df[df['연료'] == '경유'].reset_index(drop=True)

In [49]:
df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '시도',
       '시군구', '조기폐차상태코드', '조기폐차최종승인YN'],
      dtype='object')

In [50]:
df2 = df[[
    '차대번호', 
    '법정동코드', 
    '차종', 
    '용도', 
    '연료', 
    '차종유형', 
    '시도',
    '시군구', 
    '조기폐차상태코드', 
    '조기폐차최종승인YN',
]]

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'202307'

In [ ]:
df2['테이블생성일자'] = today_date

In [54]:
df3 = df2[[
    '테이블생성일자',  
    '차대번호', 
    '법정동코드', 
    '차종', 
    '용도', 
    '연료', 
    '차종유형', 
    '시도', 
    '시군구', 
    '조기폐차상태코드',
    '조기폐차최종승인YN', 
]]

In [55]:
chc_dict = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM', 
    '차대번호':'VIN', 
    '법정동코드':'STDG_CD', 
    '차종':'VHCTY_CD', 
    '용도':'PURPS_CD2', 
    '연료':'FUEL_CD', 
    '차종유형':'VHCTY_TY', 
    '시도':'CTPV', 
    '시군구':'SGG', 
    '조기폐차상태코드':'ELPDSRC_STTS_CD',
    '조기폐차최종승인YN':'ELPDSRC_LST_APRV_YN', 
}

In [56]:
STD_BD_GRD4_ELPDSRC_CURSTT = df3.rename(columns=chc_dict)
STD_BD_GRD4_ELPDSRC_CURSTT.columns

Index(['CRTR_YM', 'VIN', 'STDG_CD', 'VHCTY_CD', 'PURPS_CD2', 'FUEL_CD',
       'VHCTY_TY', 'CTPV', 'SGG', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'],
      dtype='object')

In [57]:
STD_BD_GRD4_ELPDSRC_CURSTT.head()

,CRTR_YM,VIN,STDG_CD,VHCTY_CD,PURPS_CD2,FUEL_CD,VHCTY_TY,CTPV,SGG,ELPDSRC_STTS_CD,ELPDSRC_LST_APRV_YN
0,202306,KMCLK18KP8C035195,4824034000,화물,개인용,경유,대형,경상남도,사천시,NaN,NaN
1,202306,KMFLA18KP8C031274,4482533000,화물,개인용,경유,대형,충청남도,태안군,NaN,NaN
2,202306,KMCGK17HP9C112030,4518010400,화물,개인용,경유,중형,전라북도,정읍시,NaN,NaN
3,202306,KNCSE01429K388374,2771026522,화물,개인용,경유,소형,대구광역시,달성군,NaN,NaN
4,202306,KLACA26RD8B226753,2826011300,승용,개인용,경유,중형,인천광역시,서구,NaN,NaN


## [출력] STD_BD_GRD4_ELPDSRC_CURSTT

In [ ]:
expdf = STD_BD_GRD4_ELPDSRC_CURSTT
table_nm = 'STD_BD_GRD4_ELPDSRC_CURSTT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 5s
we.import_from_pandas(expdf, table_nm)

# code end